# Initiation du modèle pourri à améliorer pour E2

In [1]:
!pip install -U spacy

     |████████████████████████████████| 6.0 MB 5.1 MB/s 
     |████████████████████████████████| 181 kB 42.3 MB/s 
     |████████████████████████████████| 628 kB 35.3 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 10.1 MB 38.1 MB/s 
     |████████████████████████████████| 451 kB 39.6 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [2]:
!python -m spacy download fr_core_news_lg

     |████████████████████████████████| 572.9 MB 3.1 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_lg')


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports

In [1]:
import numpy as np # Calculs matriciels
import pandas as pd

import nltk 
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
nltk.download('wordnet')
nltk.download('punkt')
# lemmatizer pour récupérer les racines des mots
import json # pour obtenir des fonctions sur des objets JSON
import pickle # Sauvegarder un objet python

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout # Éléments pour un réseau de neurones 
from tensorflow.keras.optimizers import SGD # Stochastic gradient descent il sera accéléré avec Nesterov = True

import spacy

import random

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Utilisateur\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Utilisateur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Initialisation de mes listes 

In [5]:
words=[] # Liste des mots
classes = [] # Liste des classes
documents = [] # Liste des patterns + tag associé
ignore_words = ['?', '!'] # Liste des mots à ignorer

# Import du fichier json
C'est à partir de ça qu'on va créer la personnalité de notre chatbot

In [6]:
data_file = open('/content/drive/Shareddrives/P#13_Dev IA - Ecole IA Microsoft (DRIVE APPRENANTS)/P#13_ALTERNANCE/ChatBot Simplon C14/intents.json').read()

In [7]:
intents = json.loads(data_file) 
# récupération du fichier d'intents avec des patterns, leur tag correspondant et les 
# réponses qui leur correspondent

<a id="json"></a>
# Découverte des éléments

In [8]:
intents.values() # Aperçu du melf intents.json

dict_values([[{'tag': 'contact', 'patterns': ['Je peux téléphoner ?', 'numéro téléphone', 'appeler', 'appeller', 'appel', 'apel', 'apelle', "comment avoir quelqu'un ?", "comment contacter quelqu'un?", 'joindre', 'contact', 'email', 'e-mail', 'mail', 'téléphone', 'envoyer un email', 'contacter', 'responsable', 'avoir un responsable', 'chef', 'qui est respondable?', 'coordonnées'], 'responses': ["Nous privilégions les échanges par mail, n'hésitez pas à nous joindre via le formulaire de contact : https://simplon.co/contact.html"], 'context': ['']}, {'tag': 'Mode', 'patterns': ['Distanciel ou présentiel ?', 'Les formations sont elles disponibles en distanciel ?', 'Les formations sont elles disponibles en présentiel ?', 'Les formations sont elles disponibles en distanciel ?', 'Comment les formations sont-elle dispensées ?', 'Distance', 'Remote', 'Présence', 'formation par internet'], 'responses': ["Les formations peuvent être en distanciel ou présentiel selon les cas. Cette information est 

# Remplissage des dossiers de travail
`words`, `documents` et `classes`

In [9]:
for intent in intents['intents']: # pour chaque intent du fichier intents.json
    for pattern in intent['patterns']: # pour chaque pattern de chaque intent du fichier
                                        # intents.json

        #tokenize each word
        w = nltk.word_tokenize(pattern) # par exemple "Hello there" = ["hello", "there"]
        words.extend(w) # Cela ajoute chaque mot tokenizé du pattern w à la liste `words`
        #add documents in the corpus
        documents.append((w, intent['tag'])) # Le couple w et tag sont ajoutés à la liste documents
        # docs[] = [["hello", "there"],'greetings']
        # add to our classes list
        if intent['tag'] not in classes: # Si le tag n'existe pas déjà, 
            classes.append(intent['tag']) # il est ajouté à la liste des tags

In [10]:
documents

[(['Je', 'peux', 'téléphoner', '?'], 'contact'),
 (['numéro', 'téléphone'], 'contact'),
 (['appeler'], 'contact'),
 (['appeller'], 'contact'),
 (['appel'], 'contact'),
 (['apel'], 'contact'),
 (['apelle'], 'contact'),
 (['comment', 'avoir', "quelqu'un", '?'], 'contact'),
 (['comment', 'contacter', "quelqu'un", '?'], 'contact'),
 (['joindre'], 'contact'),
 (['contact'], 'contact'),
 (['email'], 'contact'),
 (['e-mail'], 'contact'),
 (['mail'], 'contact'),
 (['téléphone'], 'contact'),
 (['envoyer', 'un', 'email'], 'contact'),
 (['contacter'], 'contact'),
 (['responsable'], 'contact'),
 (['avoir', 'un', 'responsable'], 'contact'),
 (['chef'], 'contact'),
 (['qui', 'est', 'respondable', '?'], 'contact'),
 (['coordonnées'], 'contact'),
 (['Distanciel', 'ou', 'présentiel', '?'], 'Mode'),
 (['Les',
   'formations',
   'sont',
   'elles',
   'disponibles',
   'en',
   'distanciel',
   '?'],
  'Mode'),
 (['Les',
   'formations',
   'sont',
   'elles',
   'disponibles',
   'en',
   'présentiel',

In [11]:
# Je vais réutiliser ce code donc je fais une fonction:
def display_words(display = 30):
    print(f"les {display} premiers mots sont {words[:display]}")

In [12]:
# lemmaztize and lower each word and remove duplicates
# ça c'est une boucle compactée c'est équivalent à :
# for w in words: ## pour chaque mot de la liste words
#     if w not in ignore_words: ## si le mot n'est pas dans la liste des mots à ignorer 'ignore_words'
#        w = lemmatizer.lemmatize(w.lower()) # on prend la racine du mot sans majuscule
#        words.append(w) ## on ajout w dans la liste de mots 'words'
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

In [13]:
words = sorted(list(set(words))) # Classement des mots dans l'ordre alphabétiques sans doublons
display_words()

les 30 premiers mots sont ['apel', 'apelle', 'appel', 'appeler', 'appeller', 'avoir', 'bjr', 'bonjour', "c'est", 'ce', 'charge', 'chef', 'combien', 'comme', 'comment', 'contact', 'contacter', 'coordonnées', 'coûte', 'cpf', 'd', 'date', 'de', 'dif', 'dispensées', 'disponibles', 'distance', 'distanciel', 'e-mail', 'elles']


In [14]:
# On crée le bag of word
# sort/tri classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (f"Il y a {len(documents)} documents.\n")
# classes = intents
print (f"Il y a {len(classes)} classes. \ncomposées de:\n {classes}\n")
# words = all words, vocabulary
print (f"Il y a {len(words)} mots lemmatisés sans doublons. \ncomposés de :\n {words}\n")

Il y a 59 documents.

Il y a 7 classes. 
composées de:
 ['Formation', 'Identite', 'Mode', 'Prix', 'Salutations', 'contact', 'none']

Il y a 87 mots lemmatisés sans doublons. 
composés de :
 ['apel', 'apelle', 'appel', 'appeler', 'appeller', 'avoir', 'bjr', 'bonjour', "c'est", 'ce', 'charge', 'chef', 'combien', 'comme', 'comment', 'contact', 'contacter', 'coordonnées', 'coûte', 'cpf', 'd', 'date', 'de', 'dif', 'dispensées', 'disponibles', 'distance', 'distanciel', 'e-mail', 'elles', 'email', 'en', 'envoyer', 'est', 'faites', 'financement', 'financer', 'formation', 'greeting', 'hello', 'hi', 'information', 'internet', 'je', 'joindre', 'la', 'le', 'mail', 'numéro', 'ou', 'ouverture', 'par', 'peux', 'pour', 'prix', 'prochainement', 'proposez', 'présence', 'présentiel', 'puis-je', "qu'est", "qu'est-ce-que", 'que', 'quel', 'quelle', 'quelles', "quelqu'un", 'quels', 'qui', 'quoi', 'remote', 'respondable', 'responsable', 'salut', 'simplon', 'slt', 'sont', 'sont-elle', 'sur', 'téléphone', 'télé

In [13]:
# On exporte les dictionnaires
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

NameError: name 'words' is not defined

In [16]:
[lemmatizer.lemmatize(word.lower()) for word in documents[0][0] if word not in ignore_words]

['je', 'peux', 'téléphoner']

# Modélisation

In [17]:
len(words)
len(classes)

7

In [18]:
documents[0][1]

'contact'

In [19]:
# create our training data
training = [] # Ce sera la liste de données d'entraînement
# create an empty array for our output
output_empty = [0] * len(classes) # -> [0, 0, 0, 0, 0, 0, 0, 0, 0] 
                                # matrice qui identifiera le tag avec un nombre '1'
# training set, bag of words for each sentence
for doc in documents: # pour chaque association pattern + tag dans la liste 
                        # documents
    # un doc ressemble à : [['hi', 'there'], 'greeting']
                            # pattern , tag
                            # partie 0 , partie 1
                            # pattern , tag
    
    # list of tokenized words for the pattern
    pattern_words = doc[0] # récupération de la partie pattern d'un élément
                            # de la liste documents
    # lemmatize each word - create base word, in attempt to represent related words
    # chaque mot du pattern est mis en minuscule et on ne prend que la racine
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # create our bag of words array with 1, if word match found in current pattern    
     # INITIALISE LE SAC DE MOTS
    bag = [] # sac de mots ce sera un vecteur avec un 1 pour chaque mot 
    #présent dans le pattern à la position du mots dans words. 
    # Donc à la fin on aura un vecteur de 88 chiffres.
    
    # REMPLISSAGE DU SAC DE MOT
    # On va comparer les mots de du pattern avec les mots de 'words'
    for w in words: # Pour chaque mot de la liste 'words'
        # La ligne qui suit est équivalente à :
        # if w in pattern_words: ## Si le mot du pattern est dans 'words'
        #    bag.append(1) ## on ajoute 1 à bag
        # else:            ## sinon
        #    bag.append(0) ## on ajoute un 0
        bag.append(1) if w in pattern_words else bag.append(0) 
        # Comme il y a 88 mots dans le dictionnaire,
        # on obtient une matrice de 88 '0' et '1' avec un '1' à la position
        # de chaque mot présent.
        # Par exemple si 'hello' et 'there' ont la position 1 et 2 dans 'words'
        #  on obtient le vecteur [1,1,0,0,...,0] un 
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    # on est toujours dans la boucle doc in documents donc on repart d'un
    # vecteur vierge pour vectoriser le tag (doc[1]) (rappel la forme de doc)
    output_row = list(output_empty) 
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row]) 
    # le doc vectorisé est ajouté au training
    # par exemple [[1,1,0,0,..,0,0][1, 0, 0, 0, 0, 0, 0, 0, 0]]
# shuffle our features and turn into np.array
random.shuffle(training) # on mélange les data d'entraînement
training = np.array(training) # on met training sous forme de tableau numpy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [20]:
training[0] # affichage du  premier élément du traning

array([list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 1, 0])], dtype=object)

In [21]:
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0]) # on crée les données avec la colonne 0 (patterns)
train_y = list(training[:,1]) # on crée la target avec la colonne 1 (les tag)
print("Training data created")

Training data created


In [22]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               11264     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 7)                 455       
                                                                 
Total params: 19,975
Trainable params: 19,975
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Create model - 
# 3 couches dense + un input
# input avec 88 neurones puisque les données d'entrées font la taille de words
# un neurone par mot de words
# une couche de 128 (par exemple) et une couche de (64)
# l'OUTPUT a autant de neurones que de classes et une fonction d'activation
# softmax car c'est une classification multiclasse
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist) # sauvegarde du model pour pouvoir
# l'utiliser ailleurs sans avoir à relancer tout ce script

print("model created")

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
12/12 [==============================] - 2s 4ms/step - loss: 1.9439 - accuracy: 0.2542
Epoch 2/200
12/12 [==============================] - 0s 4ms/step - loss: 1.8075 - accuracy: 0.3898
Epoch 3/200
12/12 [==============================] - 0s 4ms/step - loss: 1.7206 - accuracy: 0.4068
Epoch 4/200
12/12 [==============================] - 0s 4ms/step - loss: 1.6286 - accuracy: 0.4237
Epoch 5/200
12/12 [==============================] - 0s 4ms/step - loss: 1.5218 - accuracy: 0.4746
Epoch 6/200
12/12 [==============================] - 0s 5ms/step - loss: 1.4183 - accuracy: 0.4915
Epoch 7/200
12/12 [==============================] - 0s 4ms/step - loss: 1.4588 - accuracy: 0.4915
Epoch 8/200
12/12 [==============================] - 0s 4ms/step - loss: 1.3373 - accuracy: 0.5085
Epoch 9/200
12/12 [==============================] - 0s 4ms/step - loss: 1.2683 - accuracy: 0.6102
Epoch 10/200
12/12 [==============================] - 0s 4ms/step - loss: 1.1345 - accuracy: 0.6271
Epoch 11/

# Réponse

In [17]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('../data/intents.json').read()) 
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [20]:
def clean_up_sentence(sentence):
    '''
    Nettoie une phrase en la tokenisant et lemmatisant
    '''
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

In [27]:
bag = [0]*9
bag

[0, 0, 0, 0, 0, 0, 0, 0, 0]

In [28]:
bag[5] = 1
bag

[0, 0, 0, 0, 0, 1, 0, 0, 0]

In [29]:
for i, w in enumerate(words):
    print (f"L'indice c'est : {i} et le mot c'est : {w}")

L'indice c'est : 0 et le mot c'est : apel
L'indice c'est : 1 et le mot c'est : apelle
L'indice c'est : 2 et le mot c'est : appel
L'indice c'est : 3 et le mot c'est : appeler
L'indice c'est : 4 et le mot c'est : appeller
L'indice c'est : 5 et le mot c'est : avoir
L'indice c'est : 6 et le mot c'est : bjr
L'indice c'est : 7 et le mot c'est : bonjour
L'indice c'est : 8 et le mot c'est : c'est
L'indice c'est : 9 et le mot c'est : ce
L'indice c'est : 10 et le mot c'est : charge
L'indice c'est : 11 et le mot c'est : chef
L'indice c'est : 12 et le mot c'est : combien
L'indice c'est : 13 et le mot c'est : comme
L'indice c'est : 14 et le mot c'est : comment
L'indice c'est : 15 et le mot c'est : contact
L'indice c'est : 16 et le mot c'est : contacter
L'indice c'est : 17 et le mot c'est : coordonnées
L'indice c'est : 18 et le mot c'est : coûte
L'indice c'est : 19 et le mot c'est : cpf
L'indice c'est : 20 et le mot c'est : d
L'indice c'est : 21 et le mot c'est : date
L'indice c'est : 22 et le mot c

In [11]:
def bow(sentence, words, show_details=True):
    '''
    renvoie la phrase vectorisée ex : -> [0,0,...,0] vecteur de 88
    '''
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  # -> [0,0,...,0] vecteur de 88
    for s in sentence_words: # Pour chaque mot dans la phrase
        for i,w in enumerate(words): # pour chaque mot du dico 'words'
            if w == s: # si le mot de ma phrase est présent dans 'words'
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1 # on récupère l'indice de ce mot dans 'words'
                # et on met 1 à cette position dans bag
                if show_details: # Si on a choisit l'option de montrer les détails
                    print (f"found in bag: {w}")
            else:# TODO AMELIORATION
                # Les mots qui n'ont pas été trouvé pourraient être récupérer pour améliorer le modèle
                # et le réntraîner avec les mots qu'il n'a pas sû reconnaître
                pass
    return(np.array(bag)) # retourne le sac de mots avec des 1 à la position de 
                            #chaque mot

In [31]:
nlp = spacy.load("fr_core_news_lg")

In [15]:
words

NameError: name 'words' is not defined

In [32]:
def bow2(sentence, words, show_details=False):
    '''
    renvoie la phrase vectorisée ex : -> [0,0,...,0] vecteur de 88
    '''
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  # -> [0,0,...,0] vecteur de 88
    for s in sentence_words: # Pour chaque mot dans la phrase
        tmp_pourcent = -3000
        suceed = False
        for i,w in enumerate(words): # pour chaque mot du dico 'words'
            if w == s: # si le mot de ma phrase est présent dans 'words'
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1 # on récupère l'indice de ce mot dans 'words'
                #Variable de succès 
                suceed=True
                # et on met 1 à cette position dans bag
                if show_details: # Si on a choisit l'option de montrer les détails
                    print (f"found in bag: {w}")
        #Si le mot n'a pas été trouvé on va cherché a trouvé une similarité grace à spacy et la fonction similarity
        if suceed == False:
              print(f"{s} le mot n'a pas été trouvé recour à Spacy")
              #On stock le mot dans une variable spécial de Spacy
              mots = nlp(s)
              #On va parcourir chacuns des mots de notre dictionnaire
              for a,b in enumerate(words):
                #On stock de façon temporaire le mot en cours 
                atmp = nlp(b)
                #On stock un pourcentage de similarité du mot en cours et le mot de chaque dictionnaire
                pourcent = mots.similarity(atmp)
                #On check et on ne garde que le plus haut pourcentage de similarité
                if pourcent > tmp_pourcent:
                  tmp_pourcent = pourcent
                  #On stock la clé 
                  key = a
                  #On stock la valeur cible
                  value = b
              #Si le pourcentage dépasse un seuil de similarity acceptable on garde le mot
              if tmp_pourcent > 0.55:
                bag[key] = 1
                if show_details:
                  print (f'{b} a été remplacer par{value} with {tmp_pourcent} %')
              else :
                print(f"Le mot n'a pas été trouvé le plus proche est {value} with {tmp_pourcent} %")
              # TODO AMELIORATION
                # Les mots qui n'ont pas été trouvé pourraient être récupérer pour améliorer le modèle
                # et le réntraîner avec les mots qu'il n'a pas sû reconnaître
                pass
    return(np.array(bag)) # retourne le sac de mots avec des 1 à la position de 
                            #chaque mot

bow renvoie ça :

In [33]:
bow("Salut quel éducation proposé vous chien", words,show_details=True)

found in bag: salut
found in bag: quel
found in bag: vous


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

In [34]:
bow2("Salut quel éducation proposé vous chien", words,show_details=True)

found in bag: salut
found in bag: quel
éducation le mot n'a pas été trouvé recour à Spacy


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


’ a été remplacer parformation with 0.8044016542835298 %
proposé le mot n'a pas été trouvé recour à Spacy
’ a été remplacer parproposez with 0.6464751165428052 %
found in bag: vous
chien le mot n'a pas été trouvé recour à Spacy
Le mot n'a pas été trouvé le plus proche est quelqu'un with 0.3153063095576261 %


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

In [35]:
test = model.predict(np.array([bow("Bonnjour!", words,show_details=True)]))
test

array([[0.00425617, 0.02402028, 0.0184801 , 0.12047875, 0.04817605,
        0.02936842, 0.75522023]], dtype=float32)

In [36]:
test = model.predict(np.array([bow2("Bonnjour", words,show_details=True)]))
test

bonnjour le mot n'a pas été trouvé recour à Spacy


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


Le mot n'a pas été trouvé le plus proche est apel with 0.0 %


array([[0.00425617, 0.02402028, 0.0184801 , 0.12047875, 0.04817605,
        0.02936842, 0.75522023]], dtype=float32)

In [37]:
len(model.predict(np.array([bow("Salut!", words,show_details=False)]))[0])

7

In [38]:
results = [[i,r] for i,r in enumerate(test[0]) if r>0.000005]

In [40]:
results

[[0, 0.0042561693],
 [1, 0.024020284],
 [2, 0.018480103],
 [3, 0.12047875],
 [4, 0.04817605],
 [5, 0.029368423],
 [6, 0.75522023]]

In [41]:
result = model.predict(np.array([bow('bonjour',words,show_details=False)]))
vmax = result.max()
np.where(result == vmax)

(array([0]), array([4]))

In [7]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0] # Récupère les probabilités des 9 tags
    ERROR_THRESHOLD = 0.25

    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD ]
    # results est les  proba qui sont supérieures au seuil 
    
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True) 
    # triés du plus grand au plus petit
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [43]:
def predict_class_boost(sentence, model):
    # filter out predictions below a threshold
    p = bow2(sentence, words,show_details=True)
    res = model.predict(np.array([p]))[0] # Récupère les probabilités des 9 tags
    ERROR_THRESHOLD = 0.25

    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD ]
    # results est les  proba qui sont supérieures au seuil 
    
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True) 
    # triés du plus grand au plus petit
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

# Test Prédiction

In [44]:
predict_class('distenciel', model)

[{'intent': 'none', 'probability': '0.75522023'}]

In [45]:
classes[4]

'Salutations'

# Création d'un Dataset de test
Pour comparer les résultats du modèle "pourri" avec le modèle contenant Spacy

In [46]:
df_test = pd.read_csv("/content/drive/Shareddrives/P#13_Dev IA - Ecole IA Microsoft (DRIVE APPRENANTS)/P#13_ALTERNANCE/ChatBot Simplon C14/Test_model.csv")
df_test

,intents,test
0,contact,Comment puis-je vous contacter ?
1,Mode,Le distanciel est il possible ?
2,Prix,Quel sont vos tarif
3,Salutations,Bonjour
4,Formation,Quel education proposé vous
5,Identite,Qui es tu ?
6,none,chien chat emma
7,contact,Quel est votre numéro de téléphone ?
8,Mode,Faut il être en présentiel ?
9,Prix,Combien coûte une formation ?


In [47]:
result = []
for i in df_test.test:
  tmp = predict_class(i, model)
  result.append(tmp)

intent = []
for i,z in enumerate(result):
  intent.append(result[i][0]['intent'])
df_test['result_bow_classique']= intent

In [48]:
result = []
for i in df_test.test:
  tmp = predict_class_boost(i, model)
  result.append(tmp)

intent = []
for i,z in enumerate(result):
  intent.append(result[i][0]['intent'])
df_test['result_bow_boosted']= intent

found in bag: comment
found in bag: puis-je
found in bag: vous
found in bag: contacter
? le mot n'a pas été trouvé recour à Spacy


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


Le mot n'a pas été trouvé le plus proche est ou with 0.3537732031277698 %
found in bag: le
found in bag: distanciel
found in bag: est
il le mot n'a pas été trouvé recour à Spacy
Le mot n'a pas été trouvé le plus proche est qu'est with 0.4646925417796394 %
possible le mot n'a pas été trouvé recour à Spacy
’ a été remplacer parpour with 0.6205444783196559 %
? le mot n'a pas été trouvé recour à Spacy
Le mot n'a pas été trouvé le plus proche est ou with 0.3537732031277698 %
found in bag: quel
found in bag: sont
found in bag: vos
tarif le mot n'a pas été trouvé recour à Spacy
’ a été remplacer parprix with 0.7057829793620415 %
found in bag: bonjour
found in bag: quel
education le mot n'a pas été trouvé recour à Spacy
’ a été remplacer parformation with 0.755679907528988 %
proposé le mot n'a pas été trouvé recour à Spacy
’ a été remplacer parproposez with 0.6464751165428052 %
found in bag: vous
found in bag: qui
e le mot n'a pas été trouvé recour à Spacy
Le mot n'a pas été trouvé le plus pro

In [49]:
df_test

,intents,test,result_bow_classique,result_bow_boosted
0,contact,Comment puis-je vous contacter ?,contact,contact
1,Mode,Le distanciel est il possible ?,Mode,Mode
2,Prix,Quel sont vos tarif,Prix,Prix
3,Salutations,Bonjour,Salutations,Salutations
4,Formation,Quel education proposé vous,Prix,Formation
5,Identite,Qui es tu ?,contact,contact
6,none,chien chat emma,none,none
7,contact,Quel est votre numéro de téléphone ?,contact,contact
8,Mode,Faut il être en présentiel ?,Mode,Mode
9,Prix,Combien coûte une formation ?,Prix,Prix


### GUI

In [4]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [5]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [2]:
#Creating GUI with tkinter
import tkinter # librairie d'interface graphique
from tkinter import *

In [21]:
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 

base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()